# LLM From Scratch
#### (By: Mark Ehab Aziz)
##### (Built under Python 3.11.4)

Graduation Project for ***Sprints***, to build a LLM (Large Language Model) from scratch without the use of any NN (Neural Network) or Transformer related libraries.

Implementation will include:
- Own Transformer from scratch

Note: NLTK will be used later for tokenization using RegEx.

# Library Imports
Importing libraries that will be used in order to implement our own LLM.

In [1]:
import pandas as pd                             # Pandas for DataFrame manipulation
import numpy as np                              # Linear Algebra and Mathematical Operations
import nltk                                     # Downloading word sets
from nltk.stem import SnowballStemmer           # Stemming
from nltk.stem import WordNetLemmatizer         # Lemmatization (Better word yields)
from nltk.corpus import stopwords               # Stopwords
from nltk.tokenize import regexp_tokenize       # Tokenization using RegEx (Regular Expressions)

In [ ]:
# Downloading external dependencies to be used
# Such as Stopwords, English-only words

# Stopwords
nltk.download('stopwords')

# English Only
nltk.download('words')

# Setting constants for both normal and stopwords in English
ENGLISH_STOPWORDS = set(stopwords.words('english'))
ENGLISH_WORDS = set(nltk.corpus.words.words())

# All About Data

## Data Import

In [ ]:
# pd readcsv

## Data Exploration

In [ ]:
# df head and stuff

## Defining What To Clean
remove newlines, tabs, etc

remove urls

remove IPs

remove special characters

remove non-english words

remove stopwords obvs

die

### Regex Patterns

## Functions and such
mOdUlArItY

# Transformer?